In [1]:
import os; 
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
# os.environ['CUDA_VISIBLE_DEVICES'] = '3'
# os.environ['NCCL_DEBUG'] = 'TRACE'
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#; os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from rosemary import jpt_setup; jpt_setup()

from transformers import AutoModelForCausalLM, AutoTokenizer

import matplotlib.pyplot as plt
import os
import random
import time
import json
import pandas as pd
import numpy as np
from jinja2 import Template

import torch
import vllm

# from inference import create_prompt_with_llama2_chat_format
from extract_pathology import vllm_generate, metrics_xr_pathologies_iou, map_to_canonical_names, get_pathology_confidence

print(torch.cuda.is_available(), torch.cuda.device_count())

!nvidia-smi

/data/vision/polina/shared_software/miniconda3/envs/cxrpe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-05-14 18:29:25,468	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/data/vision/polina/shared_software/miniconda3/envs/cxrpe/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


True 1
Failed to initialize NVML: Driver/library version mismatch
NVML library version: 535.161


In [2]:
from extract_pathology import get_argparse
from rosemary import jpt_parse_args


quantization = None
gpu_memory_utilization = .9
model_name_or_path = 'results/baselines/unsloth/llama-3-8b'; model_name = 'llama-3-8b'; max_model_len = 8192; use_chat_template = False
# model_name_or_path = 'results/baselines/google/gemma-2b'; model_name = 'gemma-2b'; max_model_len = 8192; use_chat_template = False; gpu_memory_utilization = .7

icl_example_file = 'prompts/classify_pe/examples_v1.json'
test_label_file = 'prompts/classify_pe/test_set.json'
prompt_template = 'prompts/classify_pe/prompt_icl_simple_instruct.j2'
# prompt_template = 'prompts/classify_pe/prompt_icl_simple_instruct_wreference.j2'
# prompt_template = 'prompts/classify_pe/prompt_icl_simple_instruct_wreference_wexplanation.j2'
output_dir = os.path.join('results/classify_pe_from_ct_report/', f'{model_name}')


cmd = f"""
--test_label_file {test_label_file} \
--icl_example_file {icl_example_file} \
--prompt_template {prompt_template} \
--model_name_or_path {model_name_or_path} \
--max_model_len {max_model_len} \
{'--quantization ' + quantization if quantization else ''} \
--gpu_memory_utilization {gpu_memory_utilization} \
{'--use_chat_template' if use_chat_template else ''} \
--max_tokens 256 \
--torch_dtype float16 \
--output_dir {output_dir}
"""

print(cmd)

parser = get_argparse()
args = jpt_parse_args(parser, cmd)
args



--test_label_file prompts/classify_pe/test_set.json --icl_example_file prompts/classify_pe/examples_v1.json --prompt_template prompts/classify_pe/prompt_icl_simple_instruct.j2 --model_name_or_path results/baselines/unsloth/llama-3-8b --max_model_len 8192  --gpu_memory_utilization 0.9  --max_tokens 256 --torch_dtype float16 --output_dir results/classify_pe_from_ct_report/llama-3-8b



Namespace(test_label_file='prompts/classify_pe/test_set.json', icl_example_file='prompts/classify_pe/examples_v1.json', prompt_template='prompts/classify_pe/prompt_icl_simple_instruct.j2', model_name_or_path='results/baselines/unsloth/llama-3-8b', max_model_len=8192, torch_dtype='float16', quantization=None, gpu_memory_utilization=0.9, max_tokens=256, use_chat_template=False, output_dir='results/classify_pe_from_ct_report/llama-3-8b')

In [3]:
os.makedirs(args.output_dir, exist_ok=True)

In [4]:
model = vllm.LLM(
    model=args.model_name_or_path,
    tokenizer=args.model_name_or_path,
    tokenizer_mode="auto",
    tensor_parallel_size=torch.cuda.device_count(),
    dtype=getattr(torch, args.torch_dtype) if args.torch_dtype else 'auto',
    max_model_len=args.max_model_len,
    enable_prefix_caching=True,
    quantization=args.quantization,
    gpu_memory_utilization=args.gpu_memory_utilization,
    trust_remote_code=True,
)
tokenizer = model.get_tokenizer()

WARNING 05-14 18:29:26 config.py:1011] Casting torch.bfloat16 to torch.float16.
INFO 05-14 18:29:26 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='results/baselines/unsloth/llama-3-8b', speculative_config=None, tokenizer='results/baselines/unsloth/llama-3-8b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-14 18:29:26 pynccl_utils.py:17] Failed to import NCCL library: Cannot find libnccl.so.2 in the system.
INFO 05-14 18:29:26 pynccl_utils.py:18] It is expected if you are not running on NVIDIA GPUs.
INFO 05-14 18:29:27 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-14 18:29:27 selector.py:33] Using XFormers backend.


DistBackendError: NCCL error in: ../torch/csrc/distributed/c10d/ProcessGroupNCCL.cpp:1691, unhandled system error (run with NCCL_DEBUG=INFO for details), NCCL version 2.19.3
ncclSystemError: System call (e.g. socket, malloc) or external library call failed or device error. 
Last error:
nvmlInit_v2() failed: Driver/library version mismatch

In [ ]:
with open(args.icl_example_file, 'r') as f:
    examples = json.load(f)
    
print([x['label'] for x in examples])
examples = examples[2:]
print([x['label'] for x in examples])
    
random.seed(0)
random.shuffle(examples)
print(f"#In-context examples: {len(examples)}")

with open(args.prompt_template, 'r') as f:
    prompt_template = Template(f.read())
prompt_prefix = prompt_template.render(examples=examples)
print(f"Prompt prefix (inclouding icl examples) #Tokens: {len(tokenizer(prompt_prefix)['input_ids'])}")

In [ ]:
with open(args.test_label_file, 'r') as f:
    data_true = json.load(f)
    
data_true = data_true[:20]
print(f"Test set size: {len(data_true)}")

In [7]:
prompts = []
for example in data_true:
    prompt = prompt_template.render(examples=examples + [{'report': example['report']}])
    prompts.append(prompt)
    

if args.use_chat_template:
    prompts = [{'role': 'user', 'content': x} for x in prompts]
    if 'llama-2' in args.model_name_or_path.lower():
        prompts = [create_prompt_with_llama2_chat_format(x, tokenizer) for x in prompts]
    else:
        prompts = [tokenizer.apply_chat_template(x, tokenize=False, add_generation_prompt=True) for x in prompts]

lens = [len(tokenizer(x)['input_ids']) for x in prompts]
print(max(lens))
assert(max(lens) + args.max_tokens < args.max_model_len)
    
print(prompts[0])

7188
Act as an expert radiologist, extract common pathologies from radiology reports.

Here are a few examples:

##### REPORT:
FINDINGS:



Heart and vasculature: There are bilateral extensive filling defects in the
distal right and left main pulmonary arteries extending distally into the
lobar, segmental, and subsegmental branches involving all lobes of the lungs. 
There is flattening of the interventricular septum and dilatation of the right
ventricle concerning for right heart strain.  The main pulmonary artery is
dilated measuring up to 3.8 cm.  The thoracic aorta is normal in caliber
without evidence of dissection or intramural hematoma.  The pericardium and
great vessels are within normal limits. No pericardial effusion is seen.

Axilla, hila, and mediastinum: No axillary, mediastinal, or hilar
lymphadenopathy is present.  No mediastinal mass.

Pleural spaces: No pleural effusion or pneumothorax.

Lungs/airways: Linear atelectasis in the lingula is present.  No focal
consolidatio

In [8]:
sampling_params = vllm.SamplingParams(
    temperature=0,
    max_tokens=args.max_tokens,
    stop=["#####"],
)
start = time.time()
# prefix cached after first batch is processed, so need to call generate once to calculate the prefix and cache it
# outputs = vllm_generate(prompts[:1], model, sampling_params)
# outputs = vllm_generate(prompts[:len(prompts)], model, sampling_params)
outputs = vllm_generate(prompts, model, sampling_params)
elapsed = time.time()-start

print(f'model.generate() elapsed: {elapsed:.3f} s')

Processed prompts: 100%|██████████| 20/20 [00:30<00:00,  1.52s/it]

model.generate() elapsed: 30.748 s


In [ ]:

output = """
label: positive
"""


output.strip().split('\n')



In [ ]:
l = [x for x in output.strip().split('\n')]
l = dict([x.split(':') for x in l])

In [ ]:
data = []
output_reference = True

for i, example in enumerate(data_true):
    acc = example['accession_number']

    output = outputs[i]
    try:
        # more robust parsing, e.g., ignore rows that have not finished generating
        output_eval = [eval(x.strip()) for x in output.split('- ') if x!='' and x.count('"')%2==0]
        output_eval = [x for x in output_eval if len(x)==(3 if output_reference else 2)]
        if any([len(x)!=(3 if output_reference else 2) for x in output_eval]):
            raise
        if output_reference:
            output_formatted = [{'reference': x[0], 'pathology': x[1], 'confidence': x[2]} for x in output_eval]
        else:
            output_formatted = [{'pathology': x[0], 'confidence': x[1]} for x in output_eval]
    except:
        print(f'==== output cannot be evaluated/formatted properly [{i}] '+acc+'\n')
        print(output)
        output_eval = output
        output_formatted = []

    data.append({
        'accession_number': acc,
        'report': example['report'],
        'output': output,
        'output_formatted': output_formatted,
    })
    
print(f"#Examples that cannot be parsed from model generation: {sum(x['output_formatted']==[] for x in data)}")